In [1]:
import numpy as np
from ortools.linear_solver import pywraplp

In [2]:
def create_data_model(path):
    #Store the data model of the problem
    data = {}
    with open (path, "r") as f:
        lines = f.readlines()
        line = list(map(int,lines[0].strip().split()))
        N = line[0]
        M = line[1]
        SIGMA = N + M
        K = line[2]
        p = np.zeros(2*SIGMA +1, dtype='int')
        q = np.zeros(2*SIGMA +1, dtype='int')
        Q = np.zeros(K+1, dtype='int')
        d = np.zeros((2*SIGMA +1, 2*SIGMA+1), dtype='int')

        line = list(map(int,lines[1].strip().split()))
        for m in range(1,2*SIGMA +1):
            if (m<=N):
                p[m] = 1
            elif (m> SIGMA and m <= SIGMA + N):
                p[m] = -1
            if (m > N and m <= SIGMA):
                q[m] = line[m-N-1]
            elif (m>SIGMA+N and m <=2*SIGMA):
                q[m] = line[m-SIGMA-N-1]*(-1)
            else :
                q[m] = 0
                

        line = list(map(int,lines[2].strip().split()))
        for k in range(1,K+1):
            Q[k] = line[k-1]

        for i in range (0, 2*SIGMA+1):
            line = list(map(int,lines[i+3].strip().split()))
            for j in range(0, 2*SIGMA+1):
                d[i][j] = line[j]

        data['N'] = N
        data['M'] = M
        data['q'] = q.tolist()
        data['p'] = p.tolist()
        data['Q'] = Q.tolist()
        data['d'] = d.tolist()
        data['K'] = K
        data['root'] = 0
        data['SIGMA'] = SIGMA
    return data

In [4]:
def solve(data):
    solver = pywraplp.Solver.CreateSolver('SCIP')

    N = data['N']
    M = data['M']
    SIGMA = data['SIGMA']
    K = data['K']
    p = data['p']
    q = data['q']

    ############################## VARIABLES #######################################
    x = {}
    w = {}
    r = {}
    u = {}
    for k in range(1, K +1):
        for i in range(2 * SIGMA + 1):
            for j in range(2 * SIGMA + 1):
                x[k,i,j] = solver.IntVar(0,1,f'x[{k},{i},{j}]')
            w[k,i] = solver.IntVar(max(0, data['q'][i]), min(data['Q'][k], data['Q'][k] + data['q'][i]), f'w[{k},{i}]')
            r[k,i] = solver.IntVar(-1,1, f'r[{k},{i}]')

            u[k,i] = solver.IntVar(1, 2*SIGMA, f'u[{k},{i}]')

    # target = solver.IntVar(0, solver.infinity(), 'target')

    ############################## CONSTRAINTS #######################################

    # Tong so canh cua do thi la 2M + 2N + K
    # solver.Add(2 * SIGMA == sum(x[k,i,j] for k in range(1,data['K'] + 1) for i in range(2*SIGMA +1) for j in range(2*SIGMA +1)))
    cons1 = solver.Constraint(2* SIGMA + K, 2*SIGMA +K)
    for k in range(1, K+1):
        for i in range(2*SIGMA +1):
            for j in range(2*SIGMA +1):
                cons1.SetCoefficient(x[k,i,j], 1)

    #MLZ
    for i in range(1, N+1):
        for j in range(1, N+1):
                if j != i:
                    for k in range(1, K+1):
                        solver.Add(u[k, i-1]-u[k, j-1]+ 2*SIGMA* x[k, i, j] <= N-1)


    # Moi xe deu xuat phat tu root va ket thuc tai root
    for k in range(1, K+1):
    #     solver.Add(0 == sum(x[k,i,data['root']] for i in range(2*SIGMA + 1)))
        cons2 = solver.Constraint(1,1)
        cons3 = solver.Constraint(1,1)
        for i in range(1,2*SIGMA +1):
            cons2.SetCoefficient(x[k,data['root'],i],1)
            cons3.SetCoefficient(x[k,i,data['root']],1)

    #Cac dinh khong the di tham chinh no
    for k in range(1, K+1):
        for i in range(2*SIGMA +1):
            solver.Add(x[k, i, i] == 0)


    # Moi dinh khac root deu chi duoc tham 1 lan
    for i in range(1, 2 * SIGMA +1):
        solver.Add(1 == sum(x[k, j, i] for k in range(1, K + 1 ) for j in range(2*SIGMA + 1)))
        solver.Add(1 == sum(x[k, i, j] for k in range(1, K + 1 ) for j in range(2*SIGMA + 1)))

    #Xe di vao diem i thi ra tai diem i
    for k in range(1, K+1):
        for i in range(2*SIGMA +1):
            solver.Add(sum(x[k,j,i] for j in range(2*SIGMA +1)) == sum (x[k,i,j] for j in range(2*SIGMA +1)))

    # #Don tra 1 doi tuong phai cung tai 1 xe
    # for k in range(1, K +1):
    #     for j in range(1, SIGMA+1):
    #         solver.Add(sum(x[k, i, j] for i in range(2*SIGMA + 1))
    #                 == sum(x[k, i, j + SIGMA] for i in range(2*SIGMA +1)))

    # # #Tong so nguoi tren xe tai 1 thoi diem
    # for k in range(1, K +1):
    #     for i in range(2*SIGMA +1):
    #         for j in range(2*SIGMA +1):
    #             solver.Add(r[k, j] >= r[k, i] + p[j] * x[k, i, j])

    # # #Tong so hang tai mot thoi diem
    # for k in range(1, K +1):
    #     for i in range(2*SIGMA +1):
    #         for j in range(2*SIGMA +1):
    #             solver.Add(w[k, j] >= w[k, i] + q[j] * x[k, i, j])

    objects_term = []
    # for k in range(1, K + 1):
    #     total_distance = sum(x[k, i, j] * data['d'][i][j] for i in range(2*SIGMA + 1) for j in range(2*SIGMA +1))
    #     objects_term.append(total_distance)

    target = sum(x[k, i, j] * data['d'][i][j] for k in range(1, K+1) for i in range(2*SIGMA +1) for j in range(2*SIGMA +1))

    # for object in objects_term:
    #     solver.Add(object <= target)

    solver.Minimize(target)
    status = solver.Solve()

    ############################## PRINT SOLUTION #######################################
    if status == pywraplp.Solver.FEASIBLE or status == pywraplp.Solver.OPTIMAL :
        print("aaaa")
    # print('Solution:')
        print('Objective value =', solver.Objective().Value())
        s=0
        for k in range(1, K+1):
            print(f"Xe_{k}")
            for i in range(2*SIGMA+1):
                a = []
                for j in range(2*SIGMA+1):
                    s+=x[k,i,j].solution_value()
                    a.append(x[k,i,j].solution_value())
                print(a)
        print(s)
        for k in range(1, K+1):
            print(f"-- Vehicle {k} --")
            
            l = []
            ff = 0
            start = 0
            while True and ff < 1000:
                ff +=1
                l.append(start)
                print(f"Point {start}")
                for i in range(2*SIGMA+1):
                    if x[k, start, i].solution_value() == 1:
                        print(i)
                        start = i
                        break
                    if i == 0:
                        break
                print("Point 0")
                print("++ Visiting order: {} ++".format(l + [0]))
                s = data['d'][l[-1]][0]
                for i in range(1, len(l)):
                    s += data['d'][l[i-1]][l[i]]
                print("++ Distance: {} ++\n".format(s))

In [5]:
def main():
    data = create_data_model('../../res/data.text')
    solve(data)

In [6]:
main()

aaaa
Objective value = 33.0
Xe_1
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Xe_2
[0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, -0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 